
# <어느 시점에 얼만큼 어떤 재고를 입고해야하고 어느 기간동안 이를 유지해야할지>


그래서 결국에 어떤 목차가 되어야 맞는지는 잘 모르겠다! 이지만,
지금 진행할 content0 의 목차를 고민해보자면

### Content 0 : 기간 별, 지역별 상품 수요량 확인
#### 1. fc 월별 데이터 합치기(수요의 흐름을 보고싶은 것이기 때문에)
#### 2. fc 월별 데이터 중 지역별 재고 수요와 1%정도의 관련도 없어보이는 column 삭제하기
**칼럼 코드를 한국어로 잘 바꿔서 주석만 달아놓기**
#### 3. 날짜 별로 상품 수요량 확인하기 (시각화 하기)
#### 4. 지역 별로 상품 수요량 확인하기 (시각화 하기)
#### 5. FC 별로 지역별 수요량 확인하기

### Content1 : 다른 데이터 수집하기
분명 어느 정도 데이터 분석을 했고 주제 파악을 했다면 필요한 데이터가 어떤 것인지 확인될 것임.

### ContentN : Multi variable Regression 수행하기

In [84]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import collections

### 1. csv 파일 df로 불러오기

In [73]:
fc3_df = pd.read_csv('C:/Users/mnju5/FC_3.csv',encoding ='utf-8')

C:\Users\mnju5\.conda\envs\testAI\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [74]:
fc3_df.head(3)

,CORP_ID,REF_ORD_NO,BKG_NO,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_SEQ,ITEM_CD,...,DLVSUBCLSFCD,INS_ID,INS_DATE,INS_TIME,POST_ZONE,SPLIT_EXEC_YN,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2
0,KX007,2021030365922551,KX007210303370934,7,20210303,35510,90001302,22400.0,2,134265392,...,1,iNEXBatch,20210303,131425,3.0,N,세종특별자치시,NaN,서울특별시,서대문구
1,KX007,2021030365922951,KX007210303371080,7,20210303,35521,90001441,20900.0,1,4610261043_2,...,4,iNEXBatch,20210303,131425,50.0,N,경기도,광주시,경상남도,김해시
2,KX007,2021030365924361,KX007210303371081,7,20210303,35513,90001441,37900.0,1,4936599538_1,...,0,iNEXBatch,20210303,131425,53.0,N,경기도,광주시,경상남도,거제시


In [75]:
fc3_df.shape
#32개의 요소에 대한 309352-1 개의 정보

(309352, 32)

아래의 정보에 대해 조금의 설명을 달아보자면, Dtype 의 object는 문자열이라 생각하면 되고, 지금 SHPR_ADDR_2를 제외한 모든 COLUMN들이 30만개 이상의 NON-NULL임을 확인할 수 있다.


In [76]:
fc3_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309352 entries, 0 to 309351
Data columns (total 32 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   CORP_ID             309352 non-null  object 
 1   REF_ORD_NO          309352 non-null  object 
 2   BKG_NO              309349 non-null  object 
 3   BKG_TYP             309352 non-null  int64  
 4   BKG_DATE            309352 non-null  int64  
 5   BKG_TIME            309352 non-null  int64  
 6   SHPR_CD             309352 non-null  int64  
 7   INV_AMT             305414 non-null  float64
 8   ITEM_SEQ            309352 non-null  int64  
 9   ITEM_CD             309352 non-null  object 
 10  BRAND_NM            301758 non-null  object 
 11  ITEM_QTY            309352 non-null  int64  
 12  ITEM_AMT            306284 non-null  float64
 13  IF_YN               309352 non-null  object 
 14  ORDER_CRT_DATETIME  301758 non-null  float64
 15  DLV_DV              309220 non-nul

- count : non-null인 데이터 건수
- mean : 전체 데이터의 평균값
- std : 표준편차
- min : 최솟값

In [77]:
fc3_df.describe()

,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_SEQ,ITEM_QTY,ITEM_AMT,ORDER_CRT_DATETIME,DLV_DV,INS_DATE,INS_TIME,POST_ZONE
count,309352.00000,3.093520e+05,309352.000000,3.093520e+05,3.054140e+05,309352.000000,309352.000000,3.062840e+05,3.017580e+05,309220.0,3.093520e+05,309352.000000,309220.000000
mean,7.00043,2.021032e+07,95035.831852,9.000141e+07,5.675440e+04,1.403576,2.030234,4.000405e+04,2.021032e+13,1.0,2.021032e+07,145352.291160,25.587055
std,0.02073,9.171140e+00,61630.865074,1.016592e+02,6.375390e+04,1.015545,294.231886,5.156776e+04,9.513625e+06,0.0,1.187998e+01,60230.814456,18.232241
min,7.00000,2.021030e+07,0.000000,9.000130e+07,0.000000e+00,1.000000,1.000000,0.000000e+00,2.021022e+13,1.0,2.021023e+07,4.000000,1.000000
25%,7.00000,2.021031e+07,42145.000000,9.000130e+07,1.990000e+04,1.000000,1.000000,1.335000e+04,2.021031e+13,1.0,2.021031e+07,104242.000000,10.000000
50%,7.00000,2.021032e+07,100004.000000,9.000134e+07,3.780000e+04,1.000000,1.000000,2.090000e+04,2.021032e+13,1.0,2.021032e+07,144244.000000,21.000000
75%,7.00000,2.021032e+07,133351.000000,9.000154e+07,7.160000e+04,1.000000,1.000000,4.380000e+04,2.021032e+13,1.0,2.021032e+07,201428.000000,42.000000
max,8.00000,2.021033e+07,235959.000000,9.000158e+07,4.760000e+06,35.000000,163200.000000,4.760000e+06,2.021033e+13,1.0,2.021041e+07,235637.000000,99.000000


### 2. 여러 fc 데이터 파일 하나로 합치기

In [78]:
fc4_df = pd.read_csv('C:/Users/mnju5/FC_4.csv')
fc5_df = pd.read_csv('C:/Users/mnju5/FC_5.csv')
fc6_df = pd.read_csv('C:/Users/mnju5/FC_6.csv')

C:\Users\mnju5\.conda\envs\testAI\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\mnju5\.conda\envs\testAI\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


**3월달 품목별 수요량 정렬**

In [79]:
items_count = {}
for item in fc3_df['ITEM_CD']:
    if item in items_count:
        items_count[item] +=1
    else:
        items_count[item]=1

In [97]:
sorted_items_3 = sorted([(k,v) for k,v in items_count.items()], key=lambda items_count: -items_count[1])

In [98]:
sorted_items_3

[('4658408144_1', 9101),
 ('232155349_1', 6801),
 ('4905164407', 3403),
 ('5120727656_1', 3342),
 ('5584261925_1', 2684),
 ('5404706313', 2639),
 ('5184904828', 2573),
 ('4802177320_2', 2509),
 ('2475138283', 2482),
 ('598604962-3', 2451),
 ('4810960248-1', 2279),
 ('5569305322_7', 2110),
 ('4733964360_1', 2087),
 ('232155349_5', 2081),
 ('5569305322_2', 2077),
 ('414687646_1', 2036),
 ('5002901764_2', 1920),
 ('4658408144_10', 1859),
 ('598604962-1', 1823),
 ('5569305322_1', 1821),
 ('4825890857_1', 1782),
 ('4658408144_4', 1736),
 ('4884983927', 1633),
 ('5569305322_9', 1536),
 ('5002901764_1', 1526),
 ('657383604_1', 1516),
 ('4977419343_1', 1482),
 ('658479403', 1427),
 ('3667604580', 1381),
 ('3040297639-1', 1360),
 ('4571859898-1', 1359),
 ('5584460073_1', 1355),
 ('121174432_8', 1349),
 ('5161458796', 1322),
 ('5569305322_4', 1315),
 ('5521930034_2', 1311),
 ('598604962-4', 1268),
 ('4624382111_1', 1259),
 ('5569305322_8', 1258),
 ('438888608_1', 1221),
 ('4802177320_4', 1193),


In [100]:
item_fc3_df=pd.DataFrame(sorted_items_3)
item_fc3_df

,0,1
0,4658408144_1,9101
1,232155349_1,6801
2,4905164407,3403
3,5120727656_1,3342
4,5584261925_1,2684
...,...,...
10064,A0938,1
10065,A1511,1
10066,A0722,1
10067,A1015,1


In [119]:
item_fc3_df.head(50)

,0,1
0,4658408144_1,9101
1,232155349_1,6801
2,4905164407,3403
3,5120727656_1,3342
4,5584261925_1,2684
5,5404706313,2639
6,5184904828,2573
7,4802177320_2,2509
8,2475138283,2482
9,598604962-3,2451


0이 품목 코드 1이 품목 개수

**4월달 품목별 수요량 정렬**

In [91]:
items_count = {}
for item in fc4_df['ITEM_CD']:
    if item in items_count:
        items_count[item] +=1
    else:
        items_count[item]=1

In [101]:
sorted_items_4 = sorted([(k,v) for k,v in items_count.items()], key=lambda items_count: -items_count[1])

In [102]:
sorted_items_4

[('4658408144_1', 9101),
 ('232155349_1', 6801),
 ('4905164407', 3403),
 ('5120727656_1', 3342),
 ('5584261925_1', 2684),
 ('5404706313', 2639),
 ('5184904828', 2573),
 ('4802177320_2', 2509),
 ('2475138283', 2482),
 ('598604962-3', 2451),
 ('4810960248-1', 2279),
 ('5569305322_7', 2110),
 ('4733964360_1', 2087),
 ('232155349_5', 2081),
 ('5569305322_2', 2077),
 ('414687646_1', 2036),
 ('5002901764_2', 1920),
 ('4658408144_10', 1859),
 ('598604962-1', 1823),
 ('5569305322_1', 1821),
 ('4825890857_1', 1782),
 ('4658408144_4', 1736),
 ('4884983927', 1633),
 ('5569305322_9', 1536),
 ('5002901764_1', 1526),
 ('657383604_1', 1516),
 ('4977419343_1', 1482),
 ('658479403', 1427),
 ('3667604580', 1381),
 ('3040297639-1', 1360),
 ('4571859898-1', 1359),
 ('5584460073_1', 1355),
 ('121174432_8', 1349),
 ('5161458796', 1322),
 ('5569305322_4', 1315),
 ('5521930034_2', 1311),
 ('598604962-4', 1268),
 ('4624382111_1', 1259),
 ('5569305322_8', 1258),
 ('438888608_1', 1221),
 ('4802177320_4', 1193),


In [103]:
item_fc4_df=pd.DataFrame(sorted_items_4)
item_fc4_df

,0,1
0,4658408144_1,9101
1,232155349_1,6801
2,4905164407,3403
3,5120727656_1,3342
4,5584261925_1,2684
...,...,...
10064,A0938,1
10065,A1511,1
10066,A0722,1
10067,A1015,1


In [120]:
item_fc4_df.head(50)

,0,1
0,4658408144_1,9101
1,232155349_1,6801
2,4905164407,3403
3,5120727656_1,3342
4,5584261925_1,2684
5,5404706313,2639
6,5184904828,2573
7,4802177320_2,2509
8,2475138283,2482
9,598604962-3,2451


**5월달 품목별 수요량 정렬**

In [108]:
items_count = {}
for item in fc5_df['ITEM_CD']:
    if item in items_count:
        items_count[item] +=1
    else:
        items_count[item]=1 

In [109]:
sorted_items_5 = sorted([(k,v) for k,v in items_count.items()], key=lambda items_count: -items_count[1])            

In [110]:
sorted_items_5

[('4658408144_1', 9101),
 ('232155349_1', 6801),
 ('4905164407', 3403),
 ('5120727656_1', 3342),
 ('5584261925_1', 2684),
 ('5404706313', 2639),
 ('5184904828', 2573),
 ('4802177320_2', 2509),
 ('2475138283', 2482),
 ('598604962-3', 2451),
 ('4810960248-1', 2279),
 ('5569305322_7', 2110),
 ('4733964360_1', 2087),
 ('232155349_5', 2081),
 ('5569305322_2', 2077),
 ('414687646_1', 2036),
 ('5002901764_2', 1920),
 ('4658408144_10', 1859),
 ('598604962-1', 1823),
 ('5569305322_1', 1821),
 ('4825890857_1', 1782),
 ('4658408144_4', 1736),
 ('4884983927', 1633),
 ('5569305322_9', 1536),
 ('5002901764_1', 1526),
 ('657383604_1', 1516),
 ('4977419343_1', 1482),
 ('658479403', 1427),
 ('3667604580', 1381),
 ('3040297639-1', 1360),
 ('4571859898-1', 1359),
 ('5584460073_1', 1355),
 ('121174432_8', 1349),
 ('5161458796', 1322),
 ('5569305322_4', 1315),
 ('5521930034_2', 1311),
 ('598604962-4', 1268),
 ('4624382111_1', 1259),
 ('5569305322_8', 1258),
 ('438888608_1', 1221),
 ('4802177320_4', 1193),


In [126]:
item_fc5_df=pd.DataFrame(sorted_items_5)  

In [127]:
item_fc5_df.head(50)

,0,1
0,4658408144_1,9101
1,232155349_1,6801
2,4905164407,3403
3,5120727656_1,3342
4,5584261925_1,2684
5,5404706313,2639
6,5184904828,2573
7,4802177320_2,2509
8,2475138283,2482
9,598604962-3,2451


**6월달 품목별 수요량 정렬**

In [125]:
items_count = {}
for item in fc6_df['ITEM_CD']:
    if item in items_count:
        items_count[item] +=1
    else:
        items_count[item]=1

sorted_items_6 = sorted([(k,v) for k,v in items_count.items()], key=lambda items_count: -items_count[1])        
item_fc6_df=pd.DataFrame(sorted_items_6)      

In [122]:
item_fc6_df.head(50)

,0,1
0,232155349_1,5262
1,8809528221414,4706
2,8809528221117,4560
3,5299553826_1,4499
4,8809641652409,4199
5,5184904828,3545
6,4905164407,3134
7,3681859516,3093
8,8809528221605,3011
9,4658408144_1,2838


**전체 날짜별 품목 정렬**

In [114]:
allFC_df = pd.concat([fc3_df,fc4_df, fc5_df, fc6_df])

In [115]:
allFC_df.to_csv("allFC_df.csv")

In [123]:
items_count = {}
for item in allFC_df['ITEM_CD']:
    if item in items_count:
        items_count[item] +=1
    else:
        items_count[item]=1

sorted_items_all = sorted([(k,v) for k,v in items_count.items()], key=lambda items_count: -items_count[1])        
item_fcall_df=pd.DataFrame(sorted_items_all)

In [ ]:
item_fcall_df.head(50)

- 여기까지가 날짜별 분류 -

In [44]:
allFC_df.drop(['REF_ORD_NO'],axis=1, inplace=True)#원본 데이터에서 날린거임!

3-3) BKG_NO는 대한통운 주문번호 (예약번호)이고 솔직히 다 필요없다고 생각해서 아예 column 전체를 날렸다.

In [45]:
allFC_df.drop(['BKG_NO'],axis=1, inplace=True) 

In [46]:
allFC_df

,CORP_ID,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_SEQ,ITEM_CD,BRAND_NM,ITEM_QTY,...,DLVSUBCLSFCD,INS_ID,INS_DATE,INS_TIME,POST_ZONE,SPLIT_EXEC_YN,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2
0,0,7,20210303,35510,90001302,22400.0,2,134265392,단일상품,1,...,1,iNEXBatch,20210303,131425,3.0,N,세종특별자치시,NaN,서울특별시,서대문구
1,0,7,20210303,35521,90001441,20900.0,1,4610261043_2,조합형옵션상품,1,...,4,iNEXBatch,20210303,131425,50.0,N,경기도,광주시,경상남도,김해시
2,0,7,20210303,35513,90001441,37900.0,1,4936599538_1,조합형옵션상품,1,...,0,iNEXBatch,20210303,131425,53.0,N,경기도,광주시,경상남도,거제시
3,0,7,20210303,35608,90001542,69800.0,1,3858391637_1,조합형옵션상품,1,...,1,iNEXBatch,20210303,131425,6.0,N,경기도,광주시,서울특별시,서초구
4,0,7,20210303,35551,90001341,73700.0,1,4631307460,단일상품,1,...,1,iNEXBatch,20210303,131425,10.0,N,경기도,광주시,경기도,김포시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676907,1,7,20210628,91210,90001772,NaN,16,00281,NaN,100,...,1,osulloc01,20210628,91210,10.0,N,경기도,군포시,경기도,NaN
676908,1,7,20210628,91210,90001772,NaN,17,00316,NaN,168,...,1,osulloc01,20210628,91210,10.0,N,경기도,군포시,경기도,NaN
676909,1,7,20210628,91210,90001772,NaN,18,00338,NaN,100,...,1,osulloc01,20210628,91210,10.0,N,경기도,군포시,경기도,NaN
676910,1,7,20210628,91210,90001772,NaN,19,00183,NaN,50,...,1,osulloc01,20210628,91210,10.0,N,경기도,군포시,경기도,NaN


3-4) 아무래도 날려야할게 너무 많이 보여서 한 번에 날리기로 결심했다,,
- SHPR_CD : 고객사 코드
- ITEM_CD : 품목순번
- BRAND_NM : 품목명 (판매자가 설정해놓는 것이라고 한다. 품목의 종류를 확인하기 어렵다...)

In [47]:
allFC_df.drop(['SHPR_CD','ITEM_CD','BRAND_NM'],axis=1, inplace=True)

In [48]:
print(allFC_df)

       CORP_ID  BKG_TYP  BKG_DATE  BKG_TIME  INV_AMT  ITEM_SEQ  ITEM_QTY  \
0            0        7  20210303     35510  22400.0         2         1   
1            0        7  20210303     35521  20900.0         1         1   
2            0        7  20210303     35513  37900.0         1         1   
3            0        7  20210303     35608  69800.0         1         1   
4            0        7  20210303     35551  73700.0         1         1   
...        ...      ...       ...       ...      ...       ...       ...   
676907       1        7  20210628     91210      NaN        16       100   
676908       1        7  20210628     91210      NaN        17       168   
676909       1        7  20210628     91210      NaN        18       100   
676910       1        7  20210628     91210      NaN        19        50   
676911       1        7  20210628     91210      NaN        20       100   

        ITEM_AMT IF_YN  ORDER_CRT_DATETIME  ...  DLVSUBCLSFCD     INS_ID  \
0        14

3-5) 계속 날리는데 사실 이 화면에 모든 column 항목들이 안떠서 계속 삭제하고 보고 ,, 하는 중이다. ㅋㅋㅋ
- IF_YN : 수신 여부를 확인하는 것인데 N이면 수신이 되지 않았다는 거임,,,? 어쨌든 일단은 날리고.. (뭔가 불안하니깐 inplace=False로 맞추겠음.)
- ORDER_CRT_DATETIME : 주문생성시간인데 이거 딱봐도 앞의 주문 날짜 + 주문시간한 거 합친 데이터임,, 왜 넣어놨냐고 용량아깝게;
.. 뒤에부터도 같이 삭제해야지.. 마지막 4열들은 위치 값들이니 살려놓고 
- SPLIT_EXEC_YN : 배송처별 주문분할 여부,,,,, 이거는 친구들과 함께 이야기해보구
- POST_ZONE : 권역구분 : 이것도 마찬가지,, 근데 숫자로 표시되어있네 신기하네
- INS 시리즈는 다 날린다,, 전부 입력자ID, 입력일자, 입력시간이고 아직 의미있는지 모르겠음..   
일단 이렇게만!

In [49]:
allFC_df.drop(['IF_YN','ORDER_CRT_DATETIME','INS_ID','INS_DATE','INS_TIME'],axis =1, inplace = True) 

In [50]:
print(allFC_df) 

       CORP_ID  BKG_TYP  BKG_DATE  BKG_TIME  INV_AMT  ITEM_SEQ  ITEM_QTY  \
0            0        7  20210303     35510  22400.0         2         1   
1            0        7  20210303     35521  20900.0         1         1   
2            0        7  20210303     35513  37900.0         1         1   
3            0        7  20210303     35608  69800.0         1         1   
4            0        7  20210303     35551  73700.0         1         1   
...        ...      ...       ...       ...      ...       ...       ...   
676907       1        7  20210628     91210      NaN        16       100   
676908       1        7  20210628     91210      NaN        17       168   
676909       1        7  20210628     91210      NaN        18       100   
676910       1        7  20210628     91210      NaN        19        50   
676911       1        7  20210628     91210      NaN        20       100   

        ITEM_AMT  DLV_DV      REF_ITEM_SEQ  ... DLVPREARRBRANCD  \
0        14900.0    

In [51]:
allFC_df.head(3)

,CORP_ID,BKG_TYP,BKG_DATE,BKG_TIME,INV_AMT,ITEM_SEQ,ITEM_QTY,ITEM_AMT,DLV_DV,REF_ITEM_SEQ,...,DLVPREARRBRANCD,DLVPREARREMPNICKNM,DLVCLSFCD,DLVSUBCLSFCD,POST_ZONE,SPLIT_EXEC_YN,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2
0,0,7,20210303,35510,22400.0,2,1,14900.0,1.0,2021030323885041,...,7484,C06,5Z73,1,3.0,N,세종특별자치시,NaN,서울특별시,서대문구
1,0,7,20210303,35521,20900.0,1,1,20900.0,1.0,2021030323885721,...,6427,M09,6R67,4,50.0,N,경기도,광주시,경상남도,김해시
2,0,7,20210303,35513,37900.0,1,1,37900.0,1.0,2021030323887981,...,6158,L32,9Y25,0,53.0,N,경기도,광주시,경상남도,거제시


아씨 칼럼 지워도 지워도 계속 많다.. 내일 할래,,,,,,

3-6) 마지막 날리기가 될 듯 싶다!   
- DLV_DV : 택배 구분인데 왜 float 값으로 저렇게 나와있지??
일단 지울 거는 
- ORDER_IDX : 중개업체 주문번호
- ORDER_YN : 접수여부

- DLVPREARREMPNICKNM : 배달예정사원 분류코드가 필요할까?? 일단은 지워보겠다,,

In [52]:
allFC_df= allFC_df.drop(['ORDER_IDX','ORDER_YN','DLVPREARREMPNICKNM'],axis =1, inplace = False) 

In [53]:
allFC_df.head(10)

,CORP_ID,BKG_TYP,BKG_DATE,BKG_TIME,INV_AMT,ITEM_SEQ,ITEM_QTY,ITEM_AMT,DLV_DV,REF_ITEM_SEQ,DLVPREARRBRANCD,DLVCLSFCD,DLVSUBCLSFCD,POST_ZONE,SPLIT_EXEC_YN,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2
0,0,7,20210303,35510,22400.0,2,1,14900.0,1.0,2021030323885041,7484,5Z73,1,3.0,N,세종특별자치시,NaN,서울특별시,서대문구
1,0,7,20210303,35521,20900.0,1,1,20900.0,1.0,2021030323885721,6427,6R67,4,50.0,N,경기도,광주시,경상남도,김해시
2,0,7,20210303,35513,37900.0,1,1,37900.0,1.0,2021030323887981,6158,9Y25,0,53.0,N,경기도,광주시,경상남도,거제시
3,0,7,20210303,35608,69800.0,1,1,69800.0,1.0,2021030323895281,8909,1N65,1,6.0,N,경기도,광주시,서울특별시,서초구
4,0,7,20210303,35551,73700.0,1,1,39800.0,1.0,2021030323897651,2777,3D84,1,10.0,N,경기도,광주시,경기도,김포시
5,0,7,20210303,35551,73700.0,2,1,15900.0,1.0,2021030323897661,2777,3D84,1,10.0,N,경기도,광주시,경기도,김포시
6,0,7,20210303,35551,73700.0,3,1,18000.0,1.0,2021030323897691,2777,3D84,1,10.0,N,경기도,광주시,경기도,김포시
7,0,7,20210303,35538,14900.0,1,1,14900.0,1.0,2021030323903491,M54T,6N20,1p,47.0,N,세종특별자치시,NaN,부산광역시,연제구
8,0,7,20210303,35732,19800.0,1,1,19800.0,1.0,2021030323904521,6174,6R67,1,50.0,N,경기도,광주시,경상남도,김해시
9,0,7,20210303,35557,37900.0,1,1,37900.0,1.0,2021030323912641,8777,2U45,2,4.0,N,경기도,광주시,서울특별시,성동구



오 근데 INV_AMT: 주문금액에 대해서 마지막 row 쪽 값이 Nan값이 있고 이를 버려야 함!
#### 3-7) 결손 데이터 처리하기 
-> isna(): NaN여부를 확인하는 API   
-> fillna() : NaN값을 다른 값으로 대체쓰면 됨   

In [54]:
allFC_df.isna()

,CORP_ID,BKG_TYP,BKG_DATE,BKG_TIME,INV_AMT,ITEM_SEQ,ITEM_QTY,ITEM_AMT,DLV_DV,REF_ITEM_SEQ,DLVPREARRBRANCD,DLVCLSFCD,DLVSUBCLSFCD,POST_ZONE,SPLIT_EXEC_YN,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676907,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True
676908,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True
676909,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True
676910,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True


True값은 NaN 값이 있다는 뜻.   

In [55]:
allFC_df.isna().sum() #칼럼별 NaN값 개수 확인 꺄악 다시 돌려버려서 다 0됨..ㅠㅠ

CORP_ID                 0
BKG_TYP                 0
BKG_DATE                0
BKG_TIME                0
INV_AMT             28011
ITEM_SEQ                0
ITEM_QTY                0
ITEM_AMT            18730
DLV_DV               1043
REF_ITEM_SEQ         1000
DLVPREARRBRANCD      1043
DLVCLSFCD            1043
DLVSUBCLSFCD         1150
POST_ZONE            1043
SPLIT_EXEC_YN           0
SHPR_ADDR_1             0
SHPR_ADDR_2        109251
CNEE_ADDR_1            35
CNEE_ADDR_2         35977
dtype: int64

In [56]:
allFC_df=allFC_df.dropna() #결손 데이터를 다 날리는건데,, 흐음 이게 맞을까 
#이시ㅜ키 돌리고 돌렸어ㅠㅠ

In [57]:
allFC_df.isna().sum() #이럼 다시 볼 필요없잖아

CORP_ID            0
BKG_TYP            0
BKG_DATE           0
BKG_TIME           0
INV_AMT            0
ITEM_SEQ           0
ITEM_QTY           0
ITEM_AMT           0
DLV_DV             0
REF_ITEM_SEQ       0
DLVPREARRBRANCD    0
DLVCLSFCD          0
DLVSUBCLSFCD       0
POST_ZONE          0
SPLIT_EXEC_YN      0
SHPR_ADDR_1        0
SHPR_ADDR_2        0
CNEE_ADDR_1        0
CNEE_ADDR_2        0
dtype: int64

INV_AMT, ITEM_AMT, DLV_DV, REF_ITEM_SEQ, DLVPREARRBRANCD, DLVCLSFCD, DLVSUBCLSFCD, POST_ZONE, SHPR_ADDR_2, CNEE_ADDR_1, CNEE_ADDR_2 : True 값이 있음 (즉 NaN값이 존재하고 이를 날려버림)

*allFC_df의 NaN값을 다 지워버렸는데 이 NaN값을 채워야함 -> 보통 평균값으로 채우는데,, 각 칼럼별로 dtype을 보면서 해야할 것 같으니 일단,,흐음 일단 해본다.*   
dtype 별로 구분!
- float64 : INV_AMT(주문금액), ITEM_AMT(품목금액), DLV_DV(택배구분), POST_ZONE(권역구분)
- object : REF_ITEM_SEQ(상품주문번호)-> 이거 왜 안날림;;하핳 다시 돌리기 지겨운데ㅠ, DLVPREARRBRANCD(배달예정점소코드), DLVCLSFCD(배달터미널코드), SHPR_ADDR_2(송화인 주소2), CNEE_ADDR_1(수화인 주소1), CNEE_ADDR_2(수화인 주소2)

float64 애들은 일단 택배구분이랑 권역구분 빼고는 fillna() 로 평균값으로 맞춰놔도 될 듯 싶음!
object 쪽 결손데이터는 어떻게 해야하징,,

## 이거 아직 안끝냄!

In [58]:
allFC_df['INV_AMT'].fillna(allFC_df['INV_AMT'].mean())

1          20900.0
2          37900.0
3          69800.0
4          73700.0
5          73700.0
            ...   
676885     16900.0
676886    138000.0
676887     13900.0
676891     77000.0
676892    125800.0
Name: INV_AMT, Length: 1635018, dtype: float64

In [59]:


allFC_df['ITEM_AMT'].fillna(allFC_df['ITEM_AMT'].mean())

1          20900.0
2          37900.0
3          69800.0
4          39800.0
5          15900.0
            ...   
676885     16900.0
676886    138000.0
676887     13900.0
676891     77000.0
676892    125800.0
Name: ITEM_AMT, Length: 1635018, dtype: float64


CORP_ID : dtype = object 인데 일단 이걸 int로 바꿔보기

In [60]:
allFC_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1635018 entries, 1 to 676892
Data columns (total 19 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   CORP_ID          1635018 non-null  object 
 1   BKG_TYP          1635018 non-null  int64  
 2   BKG_DATE         1635018 non-null  int64  
 3   BKG_TIME         1635018 non-null  int64  
 4   INV_AMT          1635018 non-null  float64
 5   ITEM_SEQ         1635018 non-null  int64  
 6   ITEM_QTY         1635018 non-null  int64  
 7   ITEM_AMT         1635018 non-null  float64
 8   DLV_DV           1635018 non-null  float64
 9   REF_ITEM_SEQ     1635018 non-null  object 
 10  DLVPREARRBRANCD  1635018 non-null  object 
 11  DLVCLSFCD        1635018 non-null  object 
 12  DLVSUBCLSFCD     1635018 non-null  object 
 13  POST_ZONE        1635018 non-null  float64
 14  SPLIT_EXEC_YN    1635018 non-null  object 
 15  SHPR_ADDR_1      1635018 non-null  object 
 16  SHPR_ADDR_2      16

In [61]:
allFC_df.astype({'CORP_ID':'int64'}).dtypes#으아 오버플로우 에러 뜸..멍청이었음 칼럼지정안함 당연히 오버플로 뜨지

CORP_ID              int64
BKG_TYP              int64
BKG_DATE             int64
BKG_TIME             int64
INV_AMT            float64
ITEM_SEQ             int64
ITEM_QTY             int64
ITEM_AMT           float64
DLV_DV             float64
REF_ITEM_SEQ        object
DLVPREARRBRANCD     object
DLVCLSFCD           object
DLVSUBCLSFCD        object
POST_ZONE          float64
SPLIT_EXEC_YN       object
SHPR_ADDR_1         object
SHPR_ADDR_2         object
CNEE_ADDR_1         object
CNEE_ADDR_2         object
dtype: object

일단 칼럼별 정리는 끝냈고! 여러 함수들을 적용하면서 데이터가 어떻게 분포되어있는지 파악한 뒤 FC별 수요량을 확인해보겠음!

- CORP_ID 는 곤지암을 0, 군포를 1로 바꾸었다. 대략 어디에 상품이 더 많을지(..이런식으로 말을 해도 될까 싶지만!) 확인해보려면 평균값을 알아본다.   
근데 생각보다 너무 오래걸리는데? 왜지...

CORP_ID 를 기준으로 데이터를 분류할 것이므로 내가 모든 값을 잘 바꾼건지 확인을 datatype을 통해 확인했다. (object -> int32로 바뀌었음을 확인할 수 있음.)

### 4. FC별 상품 수요량 파악하기

4-1) groupby() 적용 -> CORP_ID 기준으로! 

In [63]:
allFC_df['day'] = allFC_df['BKG_DATE'].apply(lambda x:'BKG_DATE'
allFC_df['month'] = allFC_df['BKG_DATE']

곤지암 : 1542726 
군포 : 92292
만큼의 상품품목들이 존재.
이

In [ ]:
allFC_df_groupby()
